SAMARTH BURKUL
24102C2004
ML_ASSIGNMENT_01

In [1]:
import requests
import os

# URL of the raw housing.csv file on GitHub
url = 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv'

# Define the local filename
filename = 'housing.csv'

# Check if the file already exists
if not os.path.exists(filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content to a local file
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"'{filename}' downloaded successfully.")
    else:
        print(f"Failed to download '{filename}'. Status code: {response.status_code}")
elif os.path.exists(filename):
    print(f"'{filename}' already exists.")

'housing.csv' already exists.


In [6]:
print("Missing values before handling:")
print(df.isnull().sum())

median_total_bedrooms = df['total_bedrooms'].median()
df['total_bedrooms'] = df['total_bedrooms'].fillna(median_total_bedrooms)

print("Missing values after filling 'total_bedrooms':")
print(df.isnull().sum())

# Identify categorical features
categorical_features = df.select_dtypes(include=['object']).columns
print(f"\nCategorical features identified: {list(categorical_features)}")

# Apply one-hot encoding to 'ocean_proximity' if it exists
if 'ocean_proximity' in df.columns:
    df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=False)
    print("\nDataFrame after one-hot encoding (first 5 rows):")
    print(df.head())
else:
    print("\n'ocean_proximity' column not found, likely already one-hot encoded.")

print("\nConcise summary of the DataFrame after preprocessing (info()):")
df.info()

Missing values before handling:
longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
median_house_value            0
ocean_proximity_<1H OCEAN     0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64
Missing values after filling 'total_bedrooms':
longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
median_house_value            0
ocean_proximity_<1H OCEAN     0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64

In [7]:
median_total_bedrooms = df['total_bedrooms'].median()
df['total_bedrooms'] = df['total_bedrooms'].fillna(median_total_bedrooms)

print("Missing values after filling 'total_bedrooms':")
print(df.isnull().sum())

# Identify categorical features
categorical_features = df.select_dtypes(include=['object']).columns
print(f"\nCategorical features identified: {list(categorical_features)}")

# Apply one-hot encoding to 'ocean_proximity' if it still exists
if 'ocean_proximity' in df.columns:
    df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=False)
    print("\n'ocean_proximity' column successfully one-hot encoded.")
else:
    print("\n'ocean_proximity' column not found, likely already one-hot encoded in a previous step.")

print("\nDataFrame after preprocessing (first 5 rows):")
print(df.head())

print("\nConcise summary of the DataFrame after preprocessing (info()):")
df.info()

from sklearn.preprocessing import StandardScaler

# 1. Separate the target variable `median_house_value` from the features
X = df.drop('median_house_value', axis=1)
y = df['median_house_value']

# 2. Identify all numerical feature columns in `X` that are not boolean
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns

# 3. Import StandardScaler and 4. Instantiate a StandardScaler object
scaler = StandardScaler()

# 5. Fit the StandardScaler to the identified numerical features in `X` and then transform these features
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# 6. Print the first few rows of the updated `X` DataFrame to verify the scaling
print("First 5 rows of X after feature scaling:")
print(X.head())

Missing values after filling 'total_bedrooms':
longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
median_house_value            0
ocean_proximity_<1H OCEAN     0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64

Categorical features identified: []

'ocean_proximity' column not found, likely already one-hot encoded in a previous step.

DataFrame after preprocessing (first 5 rows):
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.

In [11]:
import pandas as pd

# Display the results dictionary as a DataFrame for easy comparison
results_df = pd.DataFrame(results).T
print("\nModel Performance Comparison:")
print(results_df.round(2))


Model Performance Comparison:
                         RMSE (Train)  RMSE (Test)  MAE (Test)
Linear Regression            68433.94     70060.52    50670.74
Decision Tree Regressor          0.00     69116.03    43600.67


In [10]:
from sklearn.tree import DecisionTreeRegressor

# 2. Instantiate a DecisionTreeRegressor model.
decision_tree_reg = DecisionTreeRegressor(random_state=42)

# 3. Train the DecisionTreeRegressor model using X_train and y_train.
decision_tree_reg.fit(X_train, y_train)

# 4. Make predictions on X_train and X_test
y_train_pred_dt = decision_tree_reg.predict(X_train)
y_test_pred_dt = decision_tree_reg.predict(X_test)

# 5. Calculate the Root Mean Squared Error (RMSE) for the training set
rmse_train_dt = np.sqrt(mean_squared_error(y_train, y_train_pred_dt))

# 6. Calculate the Root Mean Squared Error (RMSE) for the test set
rmse_test_dt = np.sqrt(mean_squared_error(y_test, y_test_pred_dt))

# 7. Calculate the Mean Absolute Error (MAE) for the test set
mae_test_dt = mean_absolute_error(y_test, y_test_pred_dt);

# 8. Store these metrics in the `results` dictionary
results['Decision Tree Regressor'] = {
    'RMSE (Train)': rmse_train_dt,
    'RMSE (Test)': rmse_test_dt,
    'MAE (Test)': mae_test_dt
}

# 9. Print the calculated metrics for Decision Tree Regressor.
print(f"Decision Tree Regressor Metrics:")
print(f"  RMSE (Train): {results['Decision Tree Regressor']['RMSE (Train)']:.2f}")
print(f"  RMSE (Test): {results['Decision Tree Regressor']['RMSE (Test)']:.2f}")
print(f"  MAE (Test): {results['Decision Tree Regressor']['MAE (Test)']:.2f}")

Decision Tree Regressor Metrics:
  RMSE (Train): 0.00
  RMSE (Test): 69116.03
  MAE (Test): 43600.67


## Train and Evaluate Ridge Regression

### Subtask:
Instantiate and train a Ridge Regression model using `X_train` and `y_train`. Make predictions on both training and testing sets, then calculate and print RMSE for both sets and MAE for the testing set. Store the results in the `results` dictionary.

#### Instructions
1. Import `Ridge` from `sklearn.linear_model`.
2. Instantiate a `Ridge` model. (You can set `random_state=42` for reproducibility if desired, although Ridge regression is generally deterministic without it).
3. Train the `Ridge` model using `X_train` and `y_train`.
4. Make predictions on `X_train` and `X_test`, storing them as `y_train_pred_ridge` and `y_test_pred_ridge`, respectively.
5. Calculate the Root Mean Squared Error (RMSE) for the training set by taking the square root of `mean_squared_error` between `y_train` and `y_train_pred_ridge`. Store this as `rmse_train_ridge`.
6. Calculate the Root Mean Squared Error (RMSE) for the test set by taking the square root of `mean_squared_error` between `y_test` and `y_test_pred_ridge`. Store this as `rmse_test_ridge`.
7. Calculate the Mean Absolute Error (MAE) for the test set using `mean_absolute_error` between `y_test` and `y_test_pred_ridge`. Store this as `mae_test_ridge`.
8. Store these metrics in the `results` dictionary under the key 'Ridge Regression', using sub-keys 'RMSE (Train)', 'RMSE (Test)', and 'MAE (Test)'.
9. Print the calculated `RMSE (Train)`, `RMSE (Test)`, and `MAE (Test)` for Ridge Regression.

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Instantiate a LinearRegression model. Initialize a dictionary named `results` to store metrics for all models.
lin_reg = LinearRegression()
results = {}

# 3. Train the LinearRegression model using X_train and y_train.
lin_reg.fit(X_train, y_train)

# 4. Make predictions on X_train and X_test, storing them as y_train_pred and y_test_pred, respectively.
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

# 5. Calculate the Root Mean Squared Error (RMSE) for the training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# 6. Calculate the Root Mean Squared Error (RMSE) for the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# 7. Calculate the Mean Absolute Error (MAE) for the test set
mae_test = mean_absolute_error(y_test, y_test_pred)

# 8. Store these metrics in the `results` dictionary
results['Linear Regression'] = {
    'RMSE (Train)': rmse_train,
    'RMSE (Test)': rmse_test,
    'MAE (Test)': mae_test
}

# 9. Print the calculated metrics for Linear Regression.
print(f"Linear Regression Metrics:")
print(f"  RMSE (Train): {results['Linear Regression']['RMSE (Train)']:.2f}")
print(f"  RMSE (Test): {results['Linear Regression']['RMSE (Test)']:.2f}")
print(f"  MAE (Test): {results['Linear Regression']['MAE (Test)']:.2f}")

Linear Regression Metrics:
  RMSE (Train): 68433.94
  RMSE (Test): 70060.52
  MAE (Test): 50670.74


In [2]:
import pandas as pd

df = pd.read_csv('housing.csv')

print("First 5 rows of the DataFrame:")
print(df.head())

First 5 rows of the DataFrame:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  


In [3]:
print("\nConcise summary of the DataFrame (info()):")
df.info()

print("\nDescriptive statistics of the DataFrame (describe()):")
print(df.describe())


Concise summary of the DataFrame (info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB

Descriptive statistics of the DataFrame (describe()):
          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -1

## Model Comparison and Analysis

**Performance Comparison:**

Let's analyze the performance of the three models based on the calculated metrics:

| Model                   | RMSE (Train) | RMSE (Test) | MAE (Test) |
|:------------------------|:-------------|:------------|:-----------|
| Linear Regression       | 68433.94     | 70060.52    | 50670.74   |
| Ridge Regression        | 68435.00     | 70067.35    | 50677.17   |
| Decision Tree Regressor | 0.00         | 69116.03    | 43600.67   |

**Observations:**

1.  **Linear Regression and Ridge Regression:**
    *   Both models show very similar performance metrics for both training and testing sets. This suggests that the regularization applied by Ridge Regression (with default alpha) had a minimal effect, likely because the dataset does not have strong multicollinearity or excessively large coefficients. Their RMSE (Train) and RMSE (Test) are close, indicating a reasonable fit to the data without significant overfitting or underfitting to the training set alone. The test RMSEs are slightly higher than training RMSEs, which is expected.

2.  **Decision Tree Regressor:**
    *   **RMSE (Train): 0.00** This is a strong indicator of **overfitting**. A perfect RMSE on the training set means the model has learned the training data too well, essentially memorizing it, including noise. This typically happens with deep decision trees that are allowed to grow without restrictions (like `max_depth`).
    *   **RMSE (Test): 69116.03** While the training RMSE is 0, the test RMSE is comparable to, or slightly better than, Linear and Ridge Regression. This shows that despite severe overfitting to the training data, its generalization performance is not drastically worse than the linear models, and its MAE (Test) is even the best among the three. This could imply that there are some underlying patterns that the Decision Tree is capturing more effectively, but its lack of generalization on n

## Summary:

The task was to summarize the performance metrics for Linear Regression, Ridge Regression, and Decision Tree Regressor models, including RMSE (Train), RMSE (Test), and MAE (Test) for each, to facilitate a comparative analysis.

The performance metrics for the models are summarized as follows:

| Model                   | RMSE (Train) | RMSE (Test) | MAE (Test) |
|:------------------------|:-------------|:------------|:-----------|
| Linear Regression       | 68433.94     | 70060.52    | 50670.74   |
| Ridge Regression        | 68435.00     | 70067.35    | 50677.17   |
| Decision Tree Regressor | 0.00         | 69116.03    | 43600.67   |

### Data Analysis Key Findings

*   **Linear and Ridge Regression Performance:** Both Linear Regression and Ridge Regression models exhibited very similar performance, with RMSE (Train) at 68433.94 and 68435.00 respectively, and RMSE (Test) at 70060.52 and 70067.35 respectively. Their MAE (Test) were also comparable at 50670.74 and 50677.17. This suggests that the regularization in Ridge Regression had a minimal impact, possibly due to the dataset's characteristics.
*   **Decision Tree Regressor Overfitting:** The Decision Tree Regressor showed clear signs of severe overfitting, achieving a perfect RMSE (Train) of 0.00. This indicates it memorized the training data entirely.
*   **Decision Tree Regressor Test Performance:** Despite overfitting on the training data, the Decision Tree Regressor's RMSE (Test) was competitive at 69116.03, and it achieved the lowest MAE (Test) among all models at 43600.67, suggesting better average absolute predictions on unseen data.
*   **No Significant Underfitting:** None of the models appeared to be significantly underfitting, as indicated by reasonably close train and test scores for the linear models and competitive test scores for the Decision Tree.